In [ ]:
# 顧客流失偵測

In [ ]:
## 載入套件與讀取資料

In [ ]:
### 載入套件

In [1]:
# 載入所需套件

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
'''
圖形中有中文字型的問題
參考
https://codertw.com/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80/359974/
'''
from matplotlib.font_manager import FontProperties

han_font = FontProperties(fname=r"c:/windows/fonts/msjh.ttc", size=14) # 中文字形

In [3]:
'''
設計圖形呈現的外觀風格
'''
sns.set(style="whitegrid")

In [ ]:
### 讀取資料

In [4]:
# 讀入資料檔
df = pd.read_excel('Online Retail.xlsx')

In [ ]:
### 資料清理

In [5]:
# 去除CustomerID沒有資料的紀錄
df = df.dropna(subset=['CustomerID'])

In [6]:
from datetime import datetime

# 取出2010-12-09到2011-12-09一年之間的資料
df = df[df.InvoiceDate>=datetime(2010, 12, 9, 0, 0, 0)]

In [7]:
#取出購買紀錄(不包含取消紀錄)
df = df[df.Quantity>0] 

In [ ]:
### 一年中的顧客購買次數

In [11]:
# 將同一個顧客的發票紀錄聚集成群後，計算不重複的發票編號數量，按照數量由大到小排序
CustomerData = df.groupby("CustomerID").agg({"InvoiceNo": "nunique"})\
.reset_index().sort_values("InvoiceNo", ascending=False)

CustomerData.head(10)

,CustomerID,InvoiceNo
322,12748.0,196
1859,14911.0,194
3960,17841.0,121
554,13089.0,94
1644,14606.0,90
2152,15311.0,88
475,12971.0,83
1672,14646.0,74
785,13408.0,60
2668,16029.0,60


In [18]:
# 一年中購買次數20次或以上的常客
MajorCustomers = CustomerData.CustomerID[CustomerData.InvoiceNo>=20].values

In [23]:
# 常客的購物紀錄
mc_df = df.loc[df.CustomerID.isin(MajorCustomers), ]

In [52]:
mc_freq = mc_df[["CustomerID", "InvoiceDate"]]\
.assign(InvoiceTime=df.InvoiceDate.apply(lambda x: pd.to_datetime(x)))\
.groupby("CustomerID")\
.apply(lambda sdf: sdf.InvoiceTime.unique().reset_index()\
.rename(columns={0: "InvoiceTime"})

AttributeError: 'numpy.ndarray' object has no attribute 'diff'

In [51]:
mc_freq.InvoiceTime[1]

array(['2010-12-10T09:35:00.000000000', '2010-12-10T09:48:00.000000000',
       '2010-12-17T11:52:00.000000000', '2010-12-17T11:53:00.000000000',
       '2011-01-13T13:21:00.000000000', '2011-02-03T10:47:00.000000000',
       '2011-02-23T15:53:00.000000000', '2011-03-18T09:55:00.000000000',
       '2011-03-18T09:57:00.000000000', '2011-04-04T12:10:00.000000000',
       '2011-04-04T12:11:00.000000000', '2011-05-13T08:10:00.000000000',
       '2011-05-13T08:11:00.000000000', '2011-07-04T11:59:00.000000000',
       '2011-07-07T12:38:00.000000000', '2011-07-07T12:40:00.000000000',
       '2011-08-22T10:39:00.000000000', '2011-08-24T12:39:00.000000000',
       '2011-08-24T16:13:00.000000000', '2011-09-16T10:39:00.000000000',
       '2011-09-22T17:25:00.000000000', '2011-10-05T12:24:00.000000000',
       '2011-10-05T12:25:00.000000000', '2011-11-03T09:56:00.000000000',
       '2011-11-17T09:49:00.000000000', '2011-11-24T08:50:00.000000000',
       '2011-11-24T12:36:00.000000000', '2011-11-29

In [49]:
from  itertools import chain

mc_freq_df = pd.DataFrame({
        "CustomerID": np.repeat(mc_freq.CustomerID.values, mc_freq.InvoiceTime.str.len()),
        "InvoiceTime": list(chain.from_iterable(mc_freq.InvoiceTime))})


,CustomerID,InvoiceTime
0,12415.0,2011-01-06 11:12:00
1,12415.0,2011-01-10 09:58:00
2,12415.0,2011-02-15 09:52:00
3,12415.0,2011-03-03 10:59:00
4,12415.0,2011-04-01 14:28:00
...,...,...
3592,18172.0,2011-09-22 16:48:00
3593,18172.0,2011-11-04 11:53:00
3594,18172.0,2011-11-04 11:54:00
3595,18172.0,2011-11-09 10:27:00
